# DA623 Tutorial
## Topic: Demystifying 1D convolution
Submission by: 
**VISHAL BULCHANDANI(Roll no. 200101108)**

B.Tech.(Computer Science and Engineering)



In [13]:
import numpy as np
import scipy
import plotly
import matplotlib.pyplot
import pandas as pd


# A Probability Problem:
To introduce convolution we take a simple problem in probability:
We take a balanced die. Probability distribution of the number rolled is as follows:

| X    | 1   | 2   | 3   | 4   | 5   | 6   |
|------|-----|-----|-----|-----|-----|-----|
| p(X) | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 |

Now suppose we are rolling 2 dies:

We need to find the probability distribution of sum of the results:

We can do it via a 2D table:
| X | 1    | 2    | 3    | 4    | 5    | 6    |
|---|------|------|------|------|------|------|
| 1 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |
| 2 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |
| 3 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |
| 4 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |
| 5 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |
| 6 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |

and the distribution is:
| X      | 2    | 3    | 4    | 5    | 6    | 7    | 8    | 9    | 10   | 11   | 12   |
|--------|------|------|------|------|------|------|------|------|------|------|------|
| p(sum) | 1/36 | 2/36 | 3/36 | 4/36 | 5/36 | 6/36 | 5/36 | 4/36 | 3/36 | 2/36 | 1/36 |

one of the ways in which this could be generated is:
take 2 dices A and B and revert the probability distribution of A:
and calculate using a sliding window:

| A    | 1    | 2    | 3    | 4    | 5    | 6    |      |      |      |      |      |
|------|------|------|------|------|------|------|------|------|------|------|------|
| p(A) | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |      |      |      |      |      |
| B    |      |      |      |      |      | 6    | 5    | 4    | 3    | 2    | 1    |
| p(B) |      |      |      |      |      | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |

Here we slide the B distribution to the right and multiply the coinciding rows and add the columns to get the probability:
$$
P(6+6) = \frac{1}{6} \times \frac{1}{6} 
$$

| A    | 1    | 2    | 3    | 4    | 5    | 6    |      |      |      |      |   |
|------|------|------|------|------|------|------|------|------|------|------|---|
| p(A) | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |      |      |      |      |   |
| B    |      |      |      |      | 6    | 5    | 4    | 3    | 2    | 1    |   |
| p(B) |      |      |      |      | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |   |

$$
P(5+6) = \frac{1}{6} \times \frac{1}{6}  +  \frac{1}{6} \times \frac{1}{6} = \frac{2}{36}
$$

| A    | 1    | 2    | 3    | 4    | 5    | 6    |      |      |      |   |   |
|------|------|------|------|------|------|------|------|------|------|---|---|
| p(A) | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |      |      |      |   |   |
| B    |      |      |      | 6    | 5    | 4    | 3    | 2    | 1    |   |   |
| p(B) |      |      |      | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 | 1/36 |   |   |

$$
P(10) = \frac{1}{6} \times \frac{1}{6}  +  \frac{1}{6} \times \frac{1}{6} + \frac{1}{6} \times \frac{1}{6}= \frac{3}{36}
$$


and so on...
to calculate each of the probability.

Observe that we here we revert the array of probability and then use it as a sliding window to calculate the results.
This operation is observed in many places in mathematics and is called a **convolution**.


### What is convolution:
Take arrays A and B:

$$
\begin{split}
A & = [a_1,a_2,a_3,\ldots, a_m] \\
B & = [b_1,b_2,b_3, \ldots, b_n]
\end{split}


$$
Convolution of A and B is defined by taking the reverse of array A and then sliding it across B and calculating the sum of products.

$$
(A * B)[n] = \sum_{m=0}^{M-1} A[m] \cdot B[n - m]

$$

where $M = N_A+N_B-1$



In **Python** we have two ways of calculating the convolution of arrays:
```
1. np.convolve()
2. scipy.signals.fftconvolve()

Take the previous probability problem with a weighted dice. The probabilities are shown below:

| X    | 1   | 2   | 3   | 4   | 5   | 6   |
|------|-----|-----|-----|-----|-----|-----|
| p(X) | 0.21 | 0.24 | 0.10 | 0.15 | 0.17 | 0.13 |

We calculate the probability of sum using both convolution and the normal methods.

In [14]:
probs = np.array([0.21,0.24,0.10,0.15,0.17,0.13])

dict= {}
for i in range(len(probs)):
    for j in range(len(probs)):
        if (i+j+2) in dict:
            dict[i+j+2]+=probs[i]*probs[j]
        else:
            dict[i+j+2]=probs[i]*probs[j]



con=np.convolve(probs, probs)

for key, new_prob in zip(dict.keys(), con):
    print(f"{key:-6d}   {dict[key]:.5f}    {new_prob:.5f}")


     2   0.04410    0.04410
     3   0.10080    0.10080
     4   0.09960    0.09960
     5   0.11100    0.11100
     6   0.15340    0.15340
     7   0.16620    0.16620
     8   0.11890    0.11890
     9   0.07700    0.07700
    10   0.06790    0.06790
    11   0.04420    0.04420
    12   0.01690    0.01690


As you can see the convolution function values match the value of the probabilities desired. 
In general in fact for 2 random variables: $X$ and $Y$
if their probability distribution functions are of form $$P(X) , P(Y)$$
Then the probability distribution of $X+Y$ is given as:
$$
P(X+Y) = P(X) * P(Y)
$$
where * represents convolution.

Now lets see a simple example of convolution:


In [17]:
A = [1,2]
B= [3,4,5]

print(np.convolve(A,B))
print(scipy.signal.fftconvolve(A,B))

[ 3 10 13 10]
[ 3. 10. 13. 10.]


now as we mentioned, there are 2 ways of calculating convolution in Python:

1. np.convolve()
2. scipy.signal.fftconvolve()

Both give identical outputs but there is a difference

Let us see the difference between them.

In [23]:
A = np.random.randn(100000)
B = np.random.randn(100000)

# we time both the functions
%timeit np.convolve(A,B)
%timeit scipy.signal.fftconvolve(A,B)


3.72 s ± 174 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
33.2 ms ± 566 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As you can see there is huge difference between the speed of both the functions despite them calculating the same thing. 
This is because the np.convolve function calculates the convolution by simple elemental multiplication which has a time complexity of $$ O(n^2) $$.
WHereas the scipy.signal.fftconvolve uses the Fast Fourier Transform algorithm to calculate the convolution and has a time complexity of: $$ O(n.log(n)) $$

### Polynomial Multiplication
It is very easy to visualize convolution as a sort of polynomial multiplication.

Take example: 

$$
\begin{split} 
p(x) &= x+5 \\
q(x) &= x^2+5x+6 \\
p(x)q(x) &= (x+5) (x^2+5x+6) \\
&= x^3 + 10x^2 + 31x +30
\end{split}
$$

observe that 
A=[1,5]
B=[1,5,6]

then np.convolve(A,B) gives us the coefficients of the polynomial multiplication. 
This is used in the fftconvolve function to calculate the convolution via calculating the polynomial product through DFT.

In [24]:
A=[1,5]
B=[1,5,6]
print(np.convolve(A,B))

[ 1 10 31 30]
